In [2]:
import math
import pandas as pd
import numpy as np
import csv

In [110]:
#元データを100行ずつくらいに分けたい
def make_sparse_data(file_name):
    flux = pd.read_csv(file_name)
    cols = flux.columns.values[1:5]
    cols = np.insert(cols,0,"Time[s]")
    #print(cols)
    sparse_flux = pd.DataFrame(columns=cols)
    for i in range(len(flux.index)//100):
        flux_series = flux.iloc[i*100,0:5]
        flux_series.index = cols
        #print(flux_series)
        sparse_flux = sparse_flux.append(flux_series)
    #print(sparse_flux.head(20))
    return sparse_flux

def calcu_angle(flux_pd):
    x = flux_pd.iloc[:,1].values
    y = flux_pd.iloc[:,2].values
    z = flux_pd.iloc[:,3].values
    phi = np.arccos(z)
    #print(phi*180/np.pi)
    sin_phi = np.sin(phi)
    theta = np.arccos(x/sin_phi)
    #print(theta*180/np.pi)
    flux_direction = pd.DataFrame(columns=['time','phi','theta','shadow_func'])
    time = flux_pd.iloc[:,0].values
    phi_deg = phi*180/np.pi
    theta_deg = theta*180/np.pi
    shadow = flux_pd.iloc[:,4].values
    flux_direction = flux_direction.assign(time=time,phi=phi_deg,theta=theta_deg,shadow_func=shadow)
    #print(flux_direction)
    return flux_direction


def calcu_flux(flux_pd):
    base_flux = 1366 #[W/m2]
    shadow = flux_pd.iloc[:,3].values
    flux = shadow*1336
    flux_pd.insert(1,'flux',flux)
    #print(flux_pd)

def append_multiple_times(flux_pd,n):
    out_flux = flux_pd.iloc[:,0:4]
    #out_flux.columns = ['time[s]','flux[W/m2]','phi[deg]','theta[deg]']
    new_flux = out_flux.copy()
    for i in range(n-1):
        last_time = new_flux.iloc[len(out_flux)-1,0]
        #print(last_time)
        time = out_flux.iloc[:,0].values + last_time + 100
        #print(time)
        out_flux = out_flux.assign(time=time)
        new_flux = new_flux.append(out_flux)
    new_flux.columns = ['time[s]','flux[W/m2]','phi[deg]','theta[deg]']
    print(new_flux)
    return new_flux


def make_output_file(flux_pd,file_name):
    flux_pd.to_csv(file_name,index=None)

    


file_ = "winter/winter3.csv"
Flux_pd = make_sparse_data(file_)
angle_pd = calcu_angle(Flux_pd)
calcu_flux(angle_pd)
out_pd = append_multiple_times(angle_pd,6)
file_name = file_[0:-4] + '_flux.csv'
print(file_name)
make_output_file(out_pd,file_name)

    time[s]  flux[W/m2]   phi[deg]  theta[deg]
0       0.0      1336.0  91.840326  101.571338
1     100.0      1336.0  90.695001  101.694173
2     200.0      1336.0  89.543166  101.705453
3     300.0      1336.0  88.399380  101.606213
4     400.0      1336.0  87.266868  101.395321
..      ...         ...        ...         ...
58  37300.0      1336.0  97.542779   99.004943
59  37400.0      1336.0  96.623184   99.699270
60  37500.0      1336.0  95.636873  100.299783
61  37600.0      1336.0  94.596903  100.799546
62  37700.0      1336.0  93.517219  101.194030

[378 rows x 4 columns]
winter/winter3_flux.csv
